# Preparing files for SatScan

## Case file

In [127]:
import pandas as pd
root = "../"

Load suicide data

In [128]:
suicide = pd.read_csv(root + 'CSV/Suicide/suicide_count_08_18.csv', index_col=0)
suicide.head()

,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Total,MUNCOD
0,5,2,2,1,3,2,1,3,2,3,1,25,110001
1,0,3,2,0,3,1,1,0,2,1,2,15,110037
2,1,1,1,1,1,0,0,0,0,1,2,8,110040
3,0,0,1,1,0,2,0,1,1,0,2,8,110034
4,8,2,4,4,9,2,5,5,6,10,2,57,110002


We need to put this is the format: 
`Quantity|Year|MUNCOD`.

In [129]:
case_file_df = pd.DataFrame({})
years = list(suicide.columns)
years.remove('MUNCOD')
years.remove('Total')
population = pd.read_csv(root + 'CSV/Population/population_08_18.csv', index_col=0)
for year in years: 
        year_df = suicide[[year, "MUNCOD"]].copy()
        year_df.rename(columns={year:'Quantidade'}, inplace=True)
        year_df['Ano'] = year
        year_df = year_df.loc[year_df['MUNCOD'] % 10000 != 0]
        case_file_df = pd.concat([case_file_df, year_df])

case_file_df.head()

,Quantidade,MUNCOD,Ano
0,5,110001,2008
1,0,110037,2008
2,1,110040,2008
3,0,110034,2008
4,8,110002,2008


## Population file

In [130]:
import pandas as pd
root = "../"

Load population data

In [131]:
population = pd.read_csv(root + 'CSV/Population/population_08_18.csv', index_col=0)
population.head()

,POP_18,MUNCOD,POP_17,POP_16,POP_15,POP_14,POP_13,POP_12,POP_11,POP_10,POP_09,POP_08
0,23167,110001,25437,25506,25578,25652,25728,24069,24228,24422,24354,24577
1,106168,110002,107345,105896,104401,102860,101269,92747,91570,90354,85541,84581
2,5438,110003,6224,6289,6355,6424,6495,6132,6221,6309,6695,6777
3,84813,110004,88507,87877,87226,86556,85863,79330,78959,78601,78675,78263
4,16444,110005,17934,17959,17986,18013,18041,16852,16939,17030,16622,16784


We need to put this is the format: 
`Population|Year|MUNCOD`.

In [132]:
population_df = pd.DataFrame({})
columns = list(population.columns)
columns.remove('MUNCOD')
for col in columns:
        year = "20" + col.split("_")[1]
        year_df = population[[col, "MUNCOD"]].copy()
        year_df.rename(columns={col:'População'}, inplace=True)
        year_df['Ano'] = year
        
        population_df = pd.concat([population_df, year_df])
        
population_df.head()

,População,MUNCOD,Ano
0,23167,110001,2018
1,106168,110002,2018
2,5438,110003,2018
3,84813,110004,2018
4,16444,110005,2018


Save to csv

In [133]:
population_df.to_csv(root + "CSV/SatScan/population.csv")

Update case file to remove zero population cities

In [134]:
df = pd.merge(population_df, case_file_df, left_on=["MUNCOD", "Ano"], right_on=["MUNCOD", "Ano"])
df = df[df["População"] > 0]
df = df.drop("População", axis=1)
df.to_csv(root + "CSV/SatScan/case_file.csv")